### Reconcile Bank data with Beals data

- Check Beals cumulative payments
    - puts credits/debits into a single column and keeps track of cumulative values - a records with an 'Amount payable' 
      with a non-zero value shows records have been carried forward from previous year or are to be carried forward to
      next.
- Reconcile Bank payments against Beals
    - iterate bank data for each payment from Beals, checking for corresponding data in Beals transactions list.
    - check for Bank records not reconciled to Beals payment
    - check for Beals records with not corresponding Bank payment
    - find Beals payments that include a transaction from previous year (identified by non zero cumulative amount)
    - find Beals transactions that have not been paid for and so are carried forward to next year (reconcile=0 flag)
- Create output spreadsheets
    - included is combo ledger of bank payments received along with Beals transactions in order to check balances

In [1]:
import numpy as np
import pandas as pd

In [2]:
import time
import os.path
import shutil

def backup_file(filename):
    timestr = time.strftime('%Y%m%d-%H%M%S')
    if os.path.isfile(filename):
        shutil.copy2(filename,filename+'.bak_' + timestr)
    return filename

In [3]:
#path = 'M:/My Documents/Business/Bugisiw Ltd/TaxReturn/TaxReturn_2020-2021/LLP accounts/'
taxreturn_yr='TaxReturn_2021-2022'
path = 'E:\\dtuklaptop\\e\\Users\\Mat\\python\\data\\property\\'+taxreturn_yr+'\\LLP accounts\\'
startdatestr='2021-04-06'
enddatestr='2022-04-05'

In [4]:
# Load bank files
#df6=pd.read_csv(path+'6045_s1.csv', parse_dates=['Date'], dayfirst=True)
#df3=pd.read_csv(path+'3072_s1.csv', parse_dates=['Date'], dayfirst=True)
input_file_6045 = path + '6045_categorised.xlsx'
df6=pd.read_excel(input_file_6045,sheet_name='6045')
df6=df6[~df6.Account.isnull()]
input_file_3072 = path + '3072_categorised.xlsx'
df3=pd.read_excel(input_file_3072,sheet_name='3072')
df3=df3[~df3.Account.isnull()]

In [5]:
# Load Beals transactions
#dfB=pd.read_csv(path+'beals.csv', parse_dates=['Date Created'], dayfirst=True)
dfB=pd.read_excel(path+'beals.xlsx', sheet_name='beals', parse_dates=['Date Created'], dayfirst=True)
dfB.insert(0,'Date', dfB['Date Created'].dt.date)
dfB=dfB.sort_values(by=['ID','Date Created'])
dfB['Date'] =  pd.to_datetime(dfB.Date)
#dfB['Item Description'] = dfB.ID + ': ' + dfB['Item Description']   

In [6]:
startdate = pd.to_datetime(startdatestr).date()
enddate = pd.to_datetime(enddatestr).date()
dfB=dfB[(dfB.Date >= startdate)&(dfB.Date <= enddate)]
df6=df6[(df6.Date >= startdate)&(df6.Date <= enddate)]
df3=df3[(df3.Date >= startdate)&(df3.Date <= enddate)]

C:\Users\MSTDTP21\.conda\envs\mtpython36\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\MSTDTP21\.conda\envs\mtpython36\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  after removing the cwd from sys.path.
C:\Users\MSTDTP21\.conda\envs\mtpython36\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Comparing Series of datetimes w

### Add cumulative payments to Beals transactions
- it's easier to see missing transacations when there is a cumulative figure

In [7]:
# Create amount column
for index, row in dfB.iterrows():
    if(row['Credit Amount']>0):
        dfB.at[index,'Amount']=-round(row['Credit Amount'],2)
    elif(row['Debit Amount']>0):
        dfB.at[index,'Amount']=round(row['Debit Amount'],2)

In [8]:
# Track cumulative amounts between payments
cumamount=0.0;
lastID=''
for index, row in dfB.iterrows():
    if(row.ID != lastID):
        cumamount=row.Amount
    else:
        cumamount=cumamount + row.Amount

    dfB.at[index,'CumAmount'] = round(cumamount,2)
    if('Amount payable' in row['Item Description']):
        cumamount=0.0
    lastID=row.ID

### Reconcile Bank payments against Beals

Iterate bank data for each payment from Beals, checking for corresponding data in Beals transactions list.

Mark reconciled data in bank and reconciled Beals payments with 1, mark Beals transactions with account they were paid into.

In [9]:
from datetime import datetime, timedelta

def find_last_rent_received(payment_index, property_code, df_beals):
    rec=df_beals[(df_beals.ID==property_code)&(df_beals['Item Description'].str.contains('Rent for period'))&(df_beals.index<payment_index)]
    if not rec.empty:
        return rec['Item Description'].iloc[-1]
    else:
        return 'NOT FOUND'

def reconcile_bank_with_beals(df_bank, df_beals,bank_label):
    lookback = 5
    errorcount = 0
    for index, row in df_bank[df_bank['Memo'].str.contains("BEALS ESTATE AGENT", na=False)].iterrows():
        # Lookback 5 days for payment to appear in Beals statement
        print('Reconciling ' + str(row.Date) + ' ' + row.Memo + ' ' + str(row.Amount))

        startdate = row.Date - timedelta(days=lookback)
        enddate = row.Date
        rec=df_beals.loc[(df_beals['ID']==row.Property)&(df_beals['Debit Amount']==row.Amount)&(df_beals['Date']>=startdate)&(df_beals['Date']<=enddate)&(df_beals.ReconciledBank!=1)]
        if not rec.empty:
            print( str(rec.iloc[0].name)+' With '+ str(rec.Date.iloc[0]) + ' ' + rec['Item Description'].iloc[0] + ' ' + str(rec['Debit Amount'].iloc[0]))
            df_bank.at[index, 'ReconciledBeals']=1
            df_bank.at[index, 'Description']=find_last_rent_received(rec.iloc[0].name,row.Property,df_beals)
            df_beals.at[rec.iloc[0].name, 'ReconciledBank']=1
            df_beals.at[rec.iloc[0].name, 'ReconciledBankAccount']=bank_label
        else:
            print('CANNOT RECONCILE')
            errorcount = errorcount + 1

    print('Errors=' +  str(errorcount))

In [10]:
df6.loc[df6['Memo'].str.contains("BEALS ESTATE AGENT", na=False),'ReconciledBeals']=0
df3.loc[df3['Memo'].str.contains("BEALS ESTATE AGENT", na=False),'ReconciledBeals']=0
dfB['ReconciledBank']=np.nan
dfB['ReconciledBankAccount']=np.nan

#### Reconcile 6045 with Beals

In [11]:
reconcile_bank_with_beals(df6,dfB,6045)

Reconciling 2021-04-06 00:00:00 BEALS ESTATE AGENT    FLAT 4321 LONDON R BGC 29.71
CANNOT RECONCILE
Reconciling 2021-04-08 00:00:00 BEALS ESTATE AGENT    FLAT 412-14 ALHAMB BGC 474.9
1630 With 2021-04-06 00:00:00 Amount payable to Mr Mathew Tucker - Flat 4, 12-14 Alhambra Road, Southsea 474.9
Reconciling 2021-04-08 00:00:00 BEALS ESTATE AGENT    FLAT 4321 LONDON R BGC 107.57
1596 With 2021-04-06 00:00:00 Amount payable to Mr Mathew Tucker - Flat 4, 321 London Road, Portsmouth 107.57
Reconciling 2021-04-08 00:00:00 BEALS ESTATE AGENT    196A KINGSTON ROAD BGC 470.32
1670 With 2021-04-06 00:00:00 Amount payable to Mr Mathew Tucker - 196a Kingston Road, Portsmouth 470.32
Reconciling 2021-04-08 00:00:00 BEALS ESTATE AGENT    FLAT 712-14 ALHAMB BGC 205.92
1706 With 2021-04-06 00:00:00 Amount payable to Mr Mathew Tucker - Flat 7, 12-14 Alhambra Road, Southsea 205.92
Reconciling 2021-04-08 00:00:00 BEALS ESTATE AGENT    FLAT 3169 FAWCETT  BGC 452.0
1609 With 2021-04-06 00:00:00 Amount payable

Reconciling 2021-06-11 00:00:00 BEALS ESTATE AGENT    FLAT 1171 FAWCETT  BGC 543.6
1929 With 2021-06-09 00:00:00 Amount payable to Mr Mathew Tucker - Flat 1, 171 Fawcett Road, Southsea 543.6
Reconciling 2021-06-15 00:00:00 BEALS ESTATE AGENT    SHOP196 KINGSTON R BGC 314.6
1876 With 2021-06-11 00:00:00 Amount payable to Mr Mathew Tucker - Shop, 196 Kingston Road, Portsmouth 314.6
Reconciling 2021-06-16 00:00:00 BEALS ESTATE AGENT    FLAT 616-18 ALHAMB BGC 314.4
1912 With 2021-06-12 00:00:00 Amount payable to Mr Mathew Tucker - Flat 6, 16-18 Alhambra Road, Southsea 314.4
Reconciling 2021-06-17 00:00:00 BEALS ESTATE AGENT    FLAT 1116-18 ALHAM BGC 497.8
1919 With 2021-06-15 00:00:00 Amount payable to Mr Mathew Tucker - Flat 11, 16-18 Alhambra Road, Southsea 497.8
Reconciling 2021-06-17 00:00:00 BEALS ESTATE AGENT    FLAT 3163 FRATTON  BGC 159.2
1869 With 2021-06-15 00:00:00 Amount payable to Mr Mathew Tucker - Flat 3, 163 Fratton Road, Portsmouth 159.2
Reconciling 2021-06-17 00:00:00 BEA

Reconciling 2021-08-26 00:00:00 BEALS ESTATE AGENT    FLAT 712-14 ALHAMB BGC 388.17
2188 With 2021-08-24 00:00:00 Amount payable to Mr Mathew Tucker - Flat 7, 12-14 Alhambra Road, Southsea 388.17
Reconciling 2021-08-26 00:00:00 BEALS ESTATE AGENT    FLAT 1012-14 ALHAM BGC 387.72
2195 With 2021-08-24 00:00:00 Amount payable to Mr Mathew Tucker - Flat 10, 12-14 Alhambra Road, Southsea 387.72
Reconciling 2021-08-26 00:00:00 BEALS ESTATE AGENT    FLAT 2171 FAWCETT  BGC 317.66
2146 With 2021-08-24 00:00:00 Amount payable to Mr Mathew Tucker - Flat 2, 171 Fawcett Road, Southsea 317.66
Reconciling 2021-08-26 00:00:00 BEALS ESTATE AGENT    FLAT 3163 FRATTON  BGC 316.38
2153 With 2021-08-24 00:00:00 Amount payable to Mr Mathew Tucker - Flat 3, 163 Fratton Road, Portsmouth 316.38
Reconciling 2021-08-26 00:00:00 BEALS ESTATE AGENT    FLAT 616-18 ALHAMB BGC 214.09
2212 With 2021-08-24 00:00:00 Amount payable to Mr Mathew Tucker - Flat 6, 16-18 Alhambra Road, Southsea 214.09
Reconciling 2021-08-26 

Reconciling 2021-11-24 00:00:00 BEALS ESTATE AGENT    FLAT 1012-14 ALHAM BGC 387.28
2470 With 2021-11-22 00:00:00 Amount payable to Mr Mathew Tucker - Flat 10, 12-14 Alhambra Road, Southsea 387.28
Reconciling 2021-11-29 00:00:00 BEALS ESTATE AGENT    FLAT 1321 LONDON R BGC 446.0
2490 With 2021-11-25 00:00:00 Amount payable to Mr Mathew Tucker - Flat 1, 321 London Road, Portsmouth 446.0
Reconciling 2021-12-06 00:00:00 BEALS ESTATE AGENT    FLAT 3169 FAWCETT  BGC 452.0
2604 With 2021-12-02 00:00:00 Amount payable to Mr Mathew Tucker - Flat 3, 169 Fawcett Road, Southsea 452.0
Reconciling 2021-12-06 00:00:00 BEALS ESTATE AGENT    196A KINGSTON ROAD BGC 470.32
2618 With 2021-12-02 00:00:00 Amount payable to Mr Mathew Tucker - 196a Kingston Road, Portsmouth 470.32
Reconciling 2021-12-08 00:00:00 BEALS ESTATE AGENT    FLAT 1412-14 ALHAM BGC 452.0
2580 With 2021-12-06 00:00:00 Amount payable to Mr Mathew Tucker - Flat 14, 12-14 Alhambra Road, Southsea 452.0
Reconciling 2021-12-10 00:00:00 BEAL

2790 With 2022-02-12 00:00:00 Amount payable to Mr Mathew Tucker - Flat 6, 16-18 Alhambra Road, Southsea 449.18
Reconciling 2022-02-17 00:00:00 BEALS ESTATE AGENT    FLAT 3163 FRATTON  BGC 286.16
2808 With 2022-02-15 00:00:00 Amount payable to Mr Mathew Tucker - Flat 3, 163 Fratton Road, Portsmouth 286.16
Reconciling 2022-02-17 00:00:00 BEALS ESTATE AGENT    FLAT 1116-18 ALHAM BGC 497.8
2812 With 2022-02-15 00:00:00 Amount payable to Mr Mathew Tucker - Flat 11, 16-18 Alhambra Road, Southsea 497.8
Reconciling 2022-02-17 00:00:00 BEALS ESTATE AGENT    FLAT 712-14 ALHAMB BGC 281.1
2801 With 2022-02-15 00:00:00 Amount payable to Mr Mathew Tucker - Flat 7, 12-14 Alhambra Road, Southsea 281.1
Reconciling 2022-02-24 00:00:00 BEALS ESTATE AGENT    FLAT 1012-14 ALHAM BGC 281.54
2781 With 2022-02-22 00:00:00 Amount payable to Mr Mathew Tucker - Flat 10, 12-14 Alhambra Road, Southsea 281.54
Reconciling 2022-03-01 00:00:00 BEALS ESTATE AGENT    FLAT 1321 LONDON R BGC 446.0
2816 With 2022-02-25 00:

#### Reconcile 3072 with Beals

In [12]:
reconcile_bank_with_beals(df3,dfB,3072)

Reconciling 2021-04-08 00:00:00 BEALS ESTATE AGENT    FLAT 512-14 ALHAMB BGC 202.91
1699 With 2021-04-06 00:00:00 Amount payable to Ms Ivana Valentino - Flat 5, 12-14 Alhambra Road, Southsea 202.91
Reconciling 2021-04-08 00:00:00 BEALS ESTATE AGENT    FLAT 1612-14 ALHAM BGC 422.29
1687 With 2021-04-06 00:00:00 Amount payable to Ms Ivana Valentino - Flat 16, 12-14 Alhambra Road, Southsea 422.29
Reconciling 2021-04-09 00:00:00 BEALS ESTATE AGENT    FLAT 1216-18 ALHAM BGC 138.21
1691 With 2021-04-07 00:00:00 Amount payable to Ms Ivana Valentino - Flat 12, 16-18 Alhambra Road, Southsea 138.21
Reconciling 2021-04-09 00:00:00 BEALS ESTATE AGENT    FLAT 312-14 ALHAMB BGC 38.45
1678 With 2021-04-07 00:00:00 Amount payable to Ms Ivana Valentino - Flat 3, 12-14 Alhambra Road, Southsea 38.45
Reconciling 2021-04-21 00:00:00 BEALS ESTATE AGENT    FLAT 312-14 ALHAMB BGC 405.27
1681 With 2021-04-19 00:00:00 Amount payable to Ms Ivana Valentino - Flat 3, 12-14 Alhambra Road, Southsea 405.27
Reconcilin

2446 With 2021-10-19 00:00:00 Amount payable to Ms Ivana Valentino - Flat 12, 16-18 Alhambra Road, Southsea 456.58
Reconciling 2021-10-21 00:00:00 BEALS ESTATE AGENT    FLAT 312-14 ALHAMB BGC 474.9
2432 With 2021-10-19 00:00:00 Amount payable to Ms Ivana Valentino - Flat 3, 12-14 Alhambra Road, Southsea 474.9
Reconciling 2021-10-27 00:00:00 BEALS ESTATE AGENT    FLAT 512-14 ALHAMB BGC 409.35
2436 With 2021-10-23 00:00:00 Amount payable to Ms Ivana Valentino - Flat 5, 12-14 Alhambra Road, Southsea 409.35
Reconciling 2021-11-03 00:00:00 BEALS ESTATE AGENT    FLAT 78 ALHAMBRA R BGC 452.0
2540 With 2021-11-01 00:00:00 Amount payable to Ms Ivana Valentino - Flat 7, 8 Alhambra Road, Southsea 452.0
Reconciling 2021-11-18 00:00:00 BEALS ESTATE AGENT    FLAT 512-14 ALHAMB BGC 27.49
2551 With 2021-11-16 00:00:00 Amount payable to Ms Ivana Valentino - Flat 5, 12-14 Alhambra Road, Southsea 27.49
Reconciling 2021-11-18 00:00:00 BEALS ESTATE AGENT    FLAT 1612-14 ALHAM BGC 281.01
2558 With 2021-11-1

#### Tidy reconciliation flag

In [13]:
# Backfill reconciliation flag so we know what bank account all Beals transactions relate to
dfB['ReconciledBank'] = dfB.groupby('ID')['ReconciledBank'].bfill()
dfB['ReconciledBank'].fillna(value=0, inplace=True)
dfB['ReconciledBankAccount'] = dfB.groupby('ID')['ReconciledBankAccount'].bfill()
dfB['ReconciledBankAccount'] = dfB.groupby('ID')['ReconciledBankAccount'].ffill()

#### Error check - Bank transactions not reconciled to Payments

- check Beals data is complete if there are bank payments without a Beals record

In [14]:
df6.loc[(df6['Memo'].str.contains("BEALS ESTATE AGENT"))&(df6.ReconciledBeals==0)]

,Date,Account,Amount,Subcategory,Memo,Property,Description,Category,ReconciledBeals
1,2021-04-06,20-74-09 60458872,29.71,DIRECTDEP,BEALS ESTATE AGENT FLAT 4321 LONDON R BGC,F4321LON,NaN,BealsRentalIncome,0.0
1380,2022-02-10,20-74-09 60458872,420.96,DIRECTDEP,BEALS ESTATE AGENT FLAT 1169 FAWCETT BGC,F1169FAW,NaN,BealsRentalIncome,0.0


In [15]:
df3.loc[(df3['Memo'].str.contains("BEALS ESTATE AGENT"))&(df3.ReconciledBeals==0)]

,Date,Account,Amount,Subcategory,Memo,Property,Description,Category,ReconciledBeals


#### Error check - Payments not reconciled to Bank transaction
- there could be Beals payments at end of year which were paid into bank after year end
- need to contact Beals if there payments on other dates 

In [16]:
dfB.loc[(dfB['Item Description'].str.contains("Amount payable"))&(dfB['Item Description'].str.contains("Tucker"))&(dfB.ReconciledBank==0)]

,Date,ID,Activity I D,Date Created,Item Description,Document Type,Document Number,Debit Amount,Credit Amount,ID Check,Amount,CumAmount,ReconciledBank,ReconciledBankAccount
3032,2022-04-05,196AKIN,879668,2022-04-05 16:12:00,Amount payable to Mr Mathew Tucker - 196a King...,Statement,260754,470.32,0.0,PM,470.32,0.0,0.0,6045.0


In [17]:
dfB.loc[(dfB['Item Description'].str.contains("Amount payable"))&(dfB['Item Description'].str.contains("Valentino"))&(dfB.ReconciledBank==0)]

,Date,ID,Activity I D,Date Created,Item Description,Document Type,Document Number,Debit Amount,Credit Amount,ID Check,Amount,CumAmount,ReconciledBank,ReconciledBankAccount
3060,2022-04-05,F78ALH,879668,2022-04-05 16:12:00,"Amount payable to Ms Ivana Valentino - Flat 7,...",Statement,260998,452.0,0.0,PM,452.0,0.0,0.0,3072.0


#### Payments with transactions from previous year

The dates on below transactions should all be around start of year. The cumulative amounts of £6 indicate Legislation & Compliance charges in previous period but paid for in this period. Dates later in year can indicate missing data in Beals spreadsheet so check the Beals paper statements match the Beals data.

In [18]:
dfB.loc[(dfB['Item Description'].str.contains("Amount payable"))&(dfB.CumAmount!=0)&(dfB.ReconciledBankAccount==6045)].sort_values(by=['Date'])

,Date,ID,Activity I D,Date Created,Item Description,Document Type,Document Number,Debit Amount,Credit Amount,ID Check,Amount,CumAmount,ReconciledBank,ReconciledBankAccount
1670,2021-04-06,196AKIN,799968,2021-04-06 11:58:00,Amount payable to Mr Mathew Tucker - 196a King...,Statement,228001,470.32,0.0,PM,470.32,-6.0,1.0,6045.0
1634,2021-04-06,F101214ALH,799968,2021-04-06 11:58:00,"Amount payable to Mr Mathew Tucker - Flat 10, ...",Statement,228167,208.26,0.0,PM,208.26,-6.0,1.0,6045.0
1605,2021-04-06,F1169FAW,799968,2021-04-06 11:58:00,"Amount payable to Mr Mathew Tucker - Flat 1, 1...",Statement,228164,426.57,0.0,PM,426.57,-6.0,1.0,6045.0
1641,2021-04-06,F141214ALH,799968,2021-04-06 11:58:00,"Amount payable to Mr Mathew Tucker - Flat 14, ...",Statement,228169,452.00,0.0,PM,452.00,-6.0,1.0,6045.0
1615,2021-04-06,F2171FAW,799968,2021-04-06 11:58:00,"Amount payable to Mr Mathew Tucker - Flat 2, 1...",Statement,228172,422.29,0.0,PM,422.29,-6.0,1.0,6045.0
1609,2021-04-06,F3169FAW,799968,2021-04-06 11:58:00,"Amount payable to Mr Mathew Tucker - Flat 3, 1...",Statement,228176,452.00,0.0,PM,452.00,-6.0,1.0,6045.0
1630,2021-04-06,F41214ALH,799968,2021-04-06 11:58:00,"Amount payable to Mr Mathew Tucker - Flat 4, 1...",Statement,228177,474.90,0.0,PM,474.90,-6.0,1.0,6045.0
1706,2021-04-06,F71214ALH,799968,2021-04-06 11:58:00,"Amount payable to Mr Mathew Tucker - Flat 7, 1...",Statement,228184,205.92,0.0,PM,205.92,-6.0,1.0,6045.0
1619,2021-04-07,F3163FRA,800659,2021-04-07 14:45:00,"Amount payable to Mr Mathew Tucker - Flat 3, 1...",Statement,228406,153.99,0.0,PM,153.99,-6.0,1.0,6045.0
1645,2021-04-07,F61618ALH,800659,2021-04-07 14:45:00,"Amount payable to Mr Mathew Tucker - Flat 6, 1...",Statement,228413,31.85,0.0,PM,31.85,-6.0,1.0,6045.0


In [31]:
dfB['Amount'].loc[(dfB['Item Description'].str.contains("Amount payable"))&(dfB.CumAmount!=0)&(dfB.ReconciledBankAccount==6045)].sum()

5559.3

In [20]:
# Output to file
#dfB.loc[(dfB['Item Description'].str.contains("Amount payable"))&(dfB.CumAmount!=0)&(dfB.ReconciledBankAccount==6045)].sort_values(by=['Date']).to_excel(path+'6045_prevyr.xlsx',sheet_name='6045', index=False)

In [21]:
dfB.loc[(dfB['Item Description'].str.contains("Amount payable"))&(dfB.CumAmount!=0)&(dfB.ReconciledBankAccount==3072)].sort_values(by=['Date'])

,Date,ID,Activity I D,Date Created,Item Description,Document Type,Document Number,Debit Amount,Credit Amount,ID Check,Amount,CumAmount,ReconciledBank,ReconciledBankAccount
1687,2021-04-06,F161214ALH,799968,2021-04-06 11:58:00,Amount payable to Ms Ivana Valentino - Flat 16...,Statement,228171,422.29,0.0,PM,422.29,-6.0,1.0,3072.0
1699,2021-04-06,F51214ALH,799968,2021-04-06 11:58:00,"Amount payable to Ms Ivana Valentino - Flat 5,...",Statement,228181,202.91,0.0,PM,202.91,-6.0,1.0,3072.0
1691,2021-04-07,F121618ALH,800659,2021-04-07 14:45:00,Amount payable to Ms Ivana Valentino - Flat 12...,Statement,228395,138.21,0.0,PM,138.21,-6.0,1.0,3072.0
1678,2021-04-07,F31214ALH,800659,2021-04-07 14:45:00,"Amount payable to Ms Ivana Valentino - Flat 3,...",Statement,228404,38.45,0.0,PM,38.45,-6.0,1.0,3072.0
2460,2021-10-04,F161618ALH,840840,2021-10-04 12:39:00,Amount payable to Ms Ivana Valentino - Flat 16...,Statement,244038,2843.40,0.0,PM,2843.40,-6.0,1.0,3072.0


In [32]:
dfB['Amount'].loc[(dfB['Item Description'].str.contains("Amount payable"))&(dfB.CumAmount!=0)&(dfB.ReconciledBankAccount==3072)].sum()

3645.2599999999998

In [23]:
# Output to file
#dfB.loc[(dfB['Item Description'].str.contains("Amount payable"))&(dfB.CumAmount!=0)&(dfB.ReconciledBankAccount==3072)].sort_values(by=['Date']).to_excel(path+'3072_prevyr.xlsx',sheet_name='3072', index=False)

#### Payments carried forward to next year
- The records in the middle of the year are a result of Beals charging Legal & Compliance fees for flats which we took over after the tenant moved out

In [24]:
dfB.loc[(dfB.ReconciledBank==0)&(dfB.ReconciledBankAccount==6045)].sort_values(by=['ID','Date'])

,Date,ID,Activity I D,Date Created,Item Description,Document Type,Document Number,Debit Amount,Credit Amount,ID Check,Amount,CumAmount,ReconciledBank,ReconciledBankAccount
2924,2022-03-28,196AKIN,877732,2022-03-28 00:56:00,Legislation & Compliance - 196a Kingston Road...,Invoice,344866,6.00,0.0,EX,6.00,6.00,0.0,6045.0
3030,2022-04-05,196AKIN,879405,2022-04-05 15:22:00,Rent for period 01/04/2022 - 30/04/2022 from M...,NaN,0,0.00,520.0,RT,-520.00,-514.00,0.0,6045.0
3031,2022-04-05,196AKIN,879405,2022-04-05 15:22:00,"Management Fee at 7% - 196a Kingston Road, Por...",Invoice,345817,43.68,0.0,MF,43.68,-470.32,0.0,6045.0
3032,2022-04-05,196AKIN,879668,2022-04-05 16:12:00,Amount payable to Mr Mathew Tucker - 196a King...,Statement,260754,470.32,0.0,PM,470.32,0.00,0.0,6045.0
2971,2022-03-28,F101214ALH,877732,2022-03-28 00:56:00,"Legislation & Compliance - Flat 10, 12-14 Alh...",Invoice,344781,6.00,0.0,EX,6.00,6.00,0.0,6045.0
2898,2022-03-28,F111618ALH,877732,2022-03-28 00:56:00,"Legislation & Compliance - Flat 11, 16-18 Alh...",Invoice,344828,6.00,0.0,EX,6.00,6.00,0.0,6045.0
2914,2022-03-28,F1169FAW,877732,2022-03-28 00:56:00,"Legislation & Compliance - Flat 1, 169 Fawcet...",Invoice,344846,6.00,0.0,EX,6.00,6.00,0.0,6045.0
2948,2022-03-28,F1171FAW,877732,2022-03-28 00:56:00,"Legislation & Compliance - Flat 1, 171 Fawcet...",Invoice,344849,6.00,0.0,EX,6.00,6.00,0.0,6045.0
2902,2022-03-28,F1321LON,877732,2022-03-28 00:56:00,"Legislation & Compliance - Flat 1, 321 London...",Invoice,344905,12.00,0.0,EX,12.00,12.00,0.0,6045.0
2888,2022-03-28,F141214ALH,877732,2022-03-28 00:56:00,"Legislation & Compliance - Flat 14, 12-14 Alh...",Invoice,344786,6.00,0.0,EX,6.00,6.00,0.0,6045.0


In [25]:
dfB['Amount'].loc[(dfB.ReconciledBank==0)&(dfB.ReconciledBankAccount==6045)].sum()

144.0

In [26]:
# Output to file
dfB.loc[(dfB.ReconciledBank==0)&(dfB.ReconciledBankAccount==6045)].sort_values(by=['Date']).to_excel(path+'6045_carriedfwd.xlsx',sheet_name='6045', index=False)

In [27]:
dfB.loc[(dfB.ReconciledBank==0)&(dfB.ReconciledBankAccount==3072)].sort_values(by=['ID','Date'])

,Date,ID,Activity I D,Date Created,Item Description,Document Type,Document Number,Debit Amount,Credit Amount,ID Check,Amount,CumAmount,ReconciledBank,ReconciledBankAccount
2982,2022-03-28,F121618ALH,877732,2022-03-28 00:56:00,"Legislation & Compliance - Flat 12, 16-18 Alh...",Invoice,344829,6.0,0.0,EX,6.0,6.0,0.0,3072.0
2975,2022-03-28,F161214ALH,877732,2022-03-28 00:56:00,"Legislation & Compliance - Flat 16, 12-14 Alh...",Invoice,344789,6.0,0.0,EX,6.0,6.0,0.0,3072.0
2983,2022-03-28,F161618ALH,877732,2022-03-28 00:56:00,"Legislation & Compliance - Flat 16, 16-18 Alha...",Invoice,345035,6.0,0.0,EX,6.0,6.0,0.0,3072.0
2964,2022-03-28,F51214ALH,877732,2022-03-28 00:56:00,"Legislation & Compliance - Flat 5, 12-14 Alha...",Invoice,344793,6.0,0.0,EX,6.0,6.0,0.0,3072.0
2952,2022-03-28,F78ALH,877732,2022-03-28 00:56:00,"Legislation & Compliance - Flat 7, 8 Alhambra...",Invoice,344959,6.0,0.0,EX,6.0,6.0,0.0,3072.0
3058,2022-04-05,F78ALH,879649,2022-04-05 16:08:00,Rent for period 30/03/2022 - 29/04/2022 from M...,NaN,0,0.0,500.0,RT,-500.0,-494.0,0.0,3072.0
3059,2022-04-05,F78ALH,879649,2022-04-05 16:08:00,"Management Fee at 7% - Flat 7, 8 Alhambra Road...",Invoice,346046,42.0,0.0,MF,42.0,-452.0,0.0,3072.0
3060,2022-04-05,F78ALH,879668,2022-04-05 16:12:00,"Amount payable to Ms Ivana Valentino - Flat 7,...",Statement,260998,452.0,0.0,PM,452.0,0.0,0.0,3072.0


In [28]:
dfB['Amount'].loc[(dfB.ReconciledBank==0)&(dfB.ReconciledBankAccount==3072)].sum()

24.0

In [29]:
dfB.loc[(dfB.ReconciledBank==0)&(dfB.ReconciledBankAccount==3072)].sort_values(by=['Date']).to_excel(path+'3072_carriedfwd.xlsx',sheet_name='3072', index=False)

### Output files
- save Bank files with Beals payment reconciliation flag
- save Beals files with bank reconciliation flag, bank account it was paid into and cumulative amount

In [ ]:
file6=backup_file(path+'6045_categorisedAndBealsReconciled.xlsx')
file3=backup_file(path+'3072_categorisedAndBealsReconciled.xlsx')
fileB=backup_file(path+'beals_bankReconciled.xlsx')
df6.to_excel(file6,sheet_name='6045', index=False)
df3.to_excel(file3,sheet_name='3072', index=False)
dfB.to_excel(fileB,sheet_name='beals', index=False)

### Create Beals/Bank reconciliation check
- this file can be used to check that the bank deposits minus agency fees match the Beals payments

In [ ]:
dfB.rename(columns={'ReconciledBankAccount':'BankAccount'},inplace=True)
dfB.rename(columns={'Item Description':'PostNarrative'},inplace=True)
dfB.rename(columns={'ID':'Property'},inplace=True)

In [ ]:
dfB.loc[(dfB.PostNarrative.str.contains('Rent for period'))&(dfB.BankAccount==6045),'Account']='0001'
dfB.loc[(dfB.PostNarrative.str.contains('Rent for period'))&(dfB.BankAccount==6045),'Account Description']='Rental Income'
dfB.loc[(dfB.PostNarrative.str.contains('Rent for period'))&(dfB.BankAccount==6045),'Category']='RentalIncome'
dfB.loc[(dfB.PostNarrative.str.contains('Rent for period'))&(dfB.BankAccount==3072),'Account']='0002'
dfB.loc[(dfB.PostNarrative.str.contains('Rent for period'))&(dfB.BankAccount==3072),'Account Description']='Rental Income'
dfB.loc[(dfB.PostNarrative.str.contains('Rent for period'))&(dfB.BankAccount==3072),'Category']='RentalIncome'

dfB.loc[(dfB.PostNarrative.str.contains('Legislation & Compliance'))&(dfB.BankAccount==6045),'Account']='1570'
dfB.loc[(dfB.PostNarrative.str.contains('Legislation & Compliance'))&(dfB.BankAccount==6045),'Account Description']='Management Fees'
dfB.loc[(dfB.PostNarrative.str.contains('Legislation & Compliance'))&(dfB.BankAccount==6045),'Category']='AgentFees'
dfB.loc[(dfB.PostNarrative.str.contains('Legislation & Compliance'))&(dfB.BankAccount==3072),'Account']='1570'
dfB.loc[(dfB.PostNarrative.str.contains('Legislation & Compliance'))&(dfB.BankAccount==3072),'Account Description']='Management Fees'
dfB.loc[(dfB.PostNarrative.str.contains('Legislation & Compliance'))&(dfB.BankAccount==3072),'Category']='AgentFees'

dfB.loc[(dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==6045),'Account']='1570'
dfB.loc[(dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==6045),'Account Description']='Management Fees'
dfB.loc[(dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==6045),'Category']='AgentFees'
dfB.loc[(dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==3072),'Account']='1570'
dfB.loc[(dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==3072),'Account Description']='Management Fees'
dfB.loc[(dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==3072),'Category']='AgentFees'

dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==6045),'Account']='4801'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==6045),'Account Description']='Payment'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==6045),'Category']='Payment'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==3072),'Account']='4801'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==3072),'Account Description']='Payment'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==3072),'Category']='Payment'

dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==6045),'Account']='4801'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==6045),'Account Description']='Payment'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==6045),'Category']='Payment'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==3072),'Account']='4801'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==3072),'Account Description']='Payment'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==3072),'Category']='Payment'

dfB.loc[(~dfB.PostNarrative.str.contains('Amount payable'))&(~dfB.PostNarrative.str.contains('Rent for period'))&(~dfB.PostNarrative.str.contains('Legislation & Compliance'))&(~dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==6045),'Account']='3140'
dfB.loc[(~dfB.PostNarrative.str.contains('Amount payable'))&(~dfB.PostNarrative.str.contains('Rent for period'))&(~dfB.PostNarrative.str.contains('Legislation & Compliance'))&(~dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==6045),'Account Description']='Repairs, renewals, maintenance'
dfB.loc[(~dfB.PostNarrative.str.contains('Amount payable'))&(~dfB.PostNarrative.str.contains('Rent for period'))&(~dfB.PostNarrative.str.contains('Legislation & Compliance'))&(~dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==6045),'Category']='Maintenance'
dfB.loc[(~dfB.PostNarrative.str.contains('Amount payable'))&(~dfB.PostNarrative.str.contains('Rent for period'))&(~dfB.PostNarrative.str.contains('Legislation & Compliance'))&(~dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==3072),'Account']='3140'
dfB.loc[(~dfB.PostNarrative.str.contains('Amount payable'))&(~dfB.PostNarrative.str.contains('Rent for period'))&(~dfB.PostNarrative.str.contains('Legislation & Compliance'))&(~dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==3072),'Account Description']='Repairs, renewals, maintenance'
dfB.loc[(~dfB.PostNarrative.str.contains('Amount payable'))&(~dfB.PostNarrative.str.contains('Rent for period'))&(~dfB.PostNarrative.str.contains('Legislation & Compliance'))&(~dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==3072),'Category']='Maintenance'


In [33]:
# Check nothing in Beals remains uncategorised
dfB[dfB.Category.isnull()]

,Date,Property,Activity I D,Date Created,PostNarrative,Document Type,Document Number,Debit Amount,Credit Amount,ID Check,Amount,CumAmount,ReconciledBank,BankAccount,Account,Account Description,Category
2347,2021-10-28,F1846ALH,846027,2021-10-28 00:36:00,"Legislation & Compliance - Flat 18, 4-6 Alham...",Invoice,324723,6.0,0.0,EX,6.0,6.0,0.0,NaN,NaN,NaN,NaN
2464,2021-11-28,F1846ALH,852357,2021-11-28 00:36:00,"Legislation & Compliance - Flat 18, 4-6 Alham...",Invoice,328778,6.0,0.0,EX,6.0,12.0,0.0,NaN,NaN,NaN,NaN
2570,2021-12-28,F1846ALH,858709,2021-12-28 00:36:00,"Legislation & Compliance - Flat 18, 4-6 Alham...",Invoice,332837,6.0,0.0,EX,6.0,18.0,0.0,NaN,NaN,NaN,NaN
2671,2022-01-28,F1846ALH,865664,2022-01-28 00:43:00,"Legislation & Compliance - Flat 18, 4-6 Alham...",Invoice,337009,6.0,0.0,EX,6.0,24.0,0.0,NaN,NaN,NaN,NaN
2775,2022-02-28,F1846ALH,871772,2022-02-28 01:06:00,"Legislation & Compliance - Flat 18, 4-6 Alham...",Invoice,340987,6.0,0.0,EX,6.0,30.0,0.0,NaN,NaN,NaN,NaN
2346,2021-10-28,F1946ALH,846027,2021-10-28 00:36:00,"Legislation & Compliance - Flat 19, 4-6 Alham...",Invoice,324725,6.0,0.0,EX,6.0,6.0,0.0,NaN,NaN,NaN,NaN
2463,2021-11-28,F1946ALH,852357,2021-11-28 00:36:00,"Legislation & Compliance - Flat 19, 4-6 Alham...",Invoice,328780,6.0,0.0,EX,6.0,12.0,0.0,NaN,NaN,NaN,NaN
2569,2021-12-28,F1946ALH,858709,2021-12-28 00:36:00,"Legislation & Compliance - Flat 19, 4-6 Alham...",Invoice,332839,6.0,0.0,EX,6.0,18.0,0.0,NaN,NaN,NaN,NaN
2670,2022-01-28,F1946ALH,865664,2022-01-28 00:43:00,"Legislation & Compliance - Flat 19, 4-6 Alham...",Invoice,337011,6.0,0.0,EX,6.0,24.0,0.0,NaN,NaN,NaN,NaN
2774,2022-02-28,F1946ALH,871772,2022-02-28 01:06:00,"Legislation & Compliance - Flat 19, 4-6 Alham...",Invoice,340989,6.0,0.0,EX,6.0,30.0,0.0,NaN,NaN,NaN,NaN


In [34]:
cols=['BankAccount','Date','Property','PostNarrative','Amount','Account','Account Description','Category']

In [35]:
dfGr6=dfB.loc[(dfB.PostNarrative.str.contains('Rent for period'))&(dfB.BankAccount==6045)][cols]
dfGr3=dfB.loc[(dfB.PostNarrative.str.contains('Rent for period'))&(dfB.BankAccount==3072)][cols]
dfLc6=dfB.loc[(dfB.PostNarrative.str.contains('Legislation & Compliance'))&(dfB.BankAccount==6045)][cols]
dfLc3=dfB.loc[(dfB.PostNarrative.str.contains('Legislation & Compliance'))&(dfB.BankAccount==3072)][cols]
dfMg6=dfB.loc[(dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==6045)][cols]
dfMg3=dfB.loc[(dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==3072)][cols]
dfOth6=dfB.loc[(~dfB.PostNarrative.str.contains('Amount payable'))&(~dfB.PostNarrative.str.contains('Rent for period'))&(~dfB.PostNarrative.str.contains('Legislation & Compliance'))&(~dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==6045)][cols]
dfOth3=dfB.loc[(~dfB.PostNarrative.str.contains('Amount payable'))&(~dfB.PostNarrative.str.contains('Rent for period'))&(~dfB.PostNarrative.str.contains('Legislation & Compliance'))&(~dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==3072)][cols]

In [36]:
df6.rename(columns={'Account':'BankAccount'},inplace=True)
df6.rename(columns={'Memo':'PostNarrative'},inplace=True)
df6['BankAccount']=6045
df6['Account Description']='Payment'
df6.loc[(df6.Category=='BealsRentalIncome'),'Account']='4801'

df3.rename(columns={'Account':'BankAccount'},inplace=True)
df3.rename(columns={'Memo':'PostNarrative'},inplace=True)
df3['BankAccount']=3072
df3['Account Description']='Payment'
df3.loc[(df6.Category=='BealsRentalIncome'),'Account']='4801'

In [37]:
dfBk6=df6.loc[(df6.Category=='BealsRentalIncome')][cols]
dfBk3=df3.loc[(df3.Category=='BealsRentalIncome')][cols]

In [38]:
dfAll6=pd.concat([dfGr6,dfLc6,dfMg6,dfBk6,dfOth6]).sort_values(by=['Property','Date'])
dfAll6.reset_index(drop=True,inplace=True)
dfAll3=pd.concat([dfGr3,dfLc3,dfMg3,dfBk3,dfOth3]).sort_values(by=['Property','Date'])
dfAll3.reset_index(drop=True,inplace=True)

In [39]:
journals=path + 'journals\\'

beals_output_file=backup_file(journals + '3.6.AgentFeesRentalIncomeBankings_Beals_6045.xlsx')
writer = pd.ExcelWriter(beals_output_file, engine='xlsxwriter',datetime_format='dd/mm/yyyy')
dfGr6.to_excel(writer,sheet_name='GrossRent', index=False)
dfLc6.to_excel(writer,sheet_name='Legal', index=False)
dfMg6.to_excel(writer,sheet_name='MgtFee', index=False)
dfOth6.to_excel(writer,sheet_name='Other', index=False)
dfBk6.to_excel(writer,sheet_name='Bank', index=False)
dfAll6.to_excel(writer,sheet_name='All', index=False)
writer.save()

beals_output_file=backup_file(journals + '3.6.AgentFeesRentalIncomeBankings_Beals_3072.xlsx')
writer = pd.ExcelWriter(beals_output_file, engine='xlsxwriter',datetime_format='dd/mm/yyyy')
dfGr3.to_excel(writer,sheet_name='GrossRent', index=False)
dfLc3.to_excel(writer,sheet_name='Legal', index=False)
dfMg3.to_excel(writer,sheet_name='MgtFee', index=False)
dfOth3.to_excel(writer,sheet_name='Other', index=False)
dfBk3.to_excel(writer,sheet_name='Bank', index=False)
dfAll3.to_excel(writer,sheet_name='All', index=False)
writer.save()